<a href="https://colab.research.google.com/github/mnechromancer/JUMP-exercises/blob/main/MorePySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, Row
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

In [4]:
commerce = spark.read.csv('/content/ecommerce_purchases.csv', multiLine=True, header=True)
commerce.show()

+--------------------+-----+--------+--------------------+--------------------+----------------+-----------+----------------+--------------------+--------------------+--------------------+---------------+--------+--------------+
|             Address|  Lot|AM or PM|        Browser Info|             Company|     Credit Card|CC Exp Date|CC Security Code|         CC Provider|               Email|                 Job|     IP Address|Language|Purchase Price|
+--------------------+-----+--------+--------------------+--------------------+----------------+-----------+----------------+--------------------+--------------------+--------------------+---------------+--------+--------------+
|16629 Pace Camp A...|46 in|      PM|Opera/9.56.(X11; ...|     Martinez-Herman|6011929061123406|      02/20|             900|        JCB 16 digit|   pdunlap@yahoo.com|Scientist, produc...|149.146.147.205|      el|         98.14|
|9374 Jasmine Spur...|28 rn|      PM|Opera/8.93.(Windo...|Fletcher, Richard...|33377

In [5]:
morning_purchases = commerce.where(commerce["AM or PM"] == "AM")
print(f'Number of AM purchases: {morning_purchases.count()}')
morning_purchases.show()

Number of AM purchases: 4932
+--------------------+-----+--------+--------------------+--------------------+----------------+-----------+----------------+--------------------+--------------------+--------------------+---------------+--------+--------------+
|             Address|  Lot|AM or PM|        Browser Info|             Company|     Credit Card|CC Exp Date|CC Security Code|         CC Provider|               Email|                 Job|     IP Address|Language|Purchase Price|
+--------------------+-----+--------+--------------------+--------------------+----------------+-----------+----------------+--------------------+--------------------+--------------------+---------------+--------+--------------+
|23012 Munoz Drive...|20 IE|      AM|Opera/9.58.(X11; ...|Brown, Watson and...|6011456623207998|      10/25|             678|Diners Club / Car...|christopherwright...|         Fine artist|   24.140.33.94|      es|         77.82|
|93971 Conway Caus...|96 Xt|      AM|Mozilla/5.0 (comp.

In [6]:
evening_purchases = commerce.where(commerce["AM or PM"] == "PM")
print(f'Number of PM purchases: {evening_purchases.count()}')
morning_purchases.show()

Number of PM purchases: 5068
+--------------------+-----+--------+--------------------+--------------------+----------------+-----------+----------------+--------------------+--------------------+--------------------+---------------+--------+--------------+
|             Address|  Lot|AM or PM|        Browser Info|             Company|     Credit Card|CC Exp Date|CC Security Code|         CC Provider|               Email|                 Job|     IP Address|Language|Purchase Price|
+--------------------+-----+--------+--------------------+--------------------+----------------+-----------+----------------+--------------------+--------------------+--------------------+---------------+--------+--------------+
|23012 Munoz Drive...|20 IE|      AM|Opera/9.58.(X11; ...|Brown, Watson and...|6011456623207998|      10/25|             678|Diners Club / Car...|christopherwright...|         Fine artist|   24.140.33.94|      es|         77.82|
|93971 Conway Caus...|96 Xt|      AM|Mozilla/5.0 (comp.

In [7]:
def group(column_name):
  print(f"Grouping by {column_name}")
  column = commerce.groupby(column_name).count()
  column.show()

  column_max = column.agg({"count": "max"})
  column_max.show()

  column_min = column.agg({"count": "sum"})
  column_min.show()

In [8]:
group("Job")
group("CC Provider")

Grouping by Job
+--------------------+-----+
|                 Job|count|
+--------------------+-----+
| Retail merchandiser|   15|
|Engineer, aeronau...|   16|
|    Catering manager|   14|
| Librarian, academic|   19|
|Diplomatic Servic...|    8|
|Designer, ceramic...|   19|
|Occupational hygi...|   17|
| Early years teacher|   21|
|     Patent examiner|   12|
|English as a seco...|   18|
|Primary school te...|   15|
|Control and instr...|   18|
|Clinical molecula...|   11|
|        Estate agent|   13|
|       Art therapist|   23|
|   Transport planner|   16|
|Applications deve...|   17|
|  Petroleum engineer|   21|
|Conservator, furn...|   19|
|       Stage manager|    7|
+--------------------+-----+
only showing top 20 rows

+----------+
|max(count)|
+----------+
|        31|
+----------+

+----------+
|sum(count)|
+----------+
|     10000|
+----------+

Grouping by CC Provider
+--------------------+-----+
|         CC Provider|count|
+--------------------+-----+
|       VISA 16 dig

In [9]:
cheap = commerce.where(commerce['Purchase Price'] < 10.00)
cheap.show()

+--------------------+-----+--------+--------------------+--------------------+----------------+-----------+----------------+--------------------+--------------------+--------------------+---------------+--------+--------------+
|             Address|  Lot|AM or PM|        Browser Info|             Company|     Credit Card|CC Exp Date|CC Security Code|         CC Provider|               Email|                 Job|     IP Address|Language|Purchase Price|
+--------------------+-----+--------+--------------------+--------------------+----------------+-----------+----------------+--------------------+--------------------+--------------------+---------------+--------+--------------+
|Unit 0065 Box 505...|94 vE|      PM|Mozilla/5.0 (comp...|Simpson, Williams...|    675957666125|      08/19|             699|        JCB 16 digit|amymiller@morales...|Customer service ...| 132.207.160.22|      de|          0.95|
|118 Melton Via Su...|36 bu|      PM|Mozilla/5.0 (Maci...|          Keller PLC|46036

In [10]:
mid = commerce.where(commerce['Purchase Price'] > 20.00).where(commerce['Purchase Price'] < 50.00)
mid.show()

+--------------------+-----+--------+--------------------+--------------------+----------------+-----------+----------------+--------------------+--------------------+--------------------+---------------+--------+--------------+
|             Address|  Lot|AM or PM|        Browser Info|             Company|     Credit Card|CC Exp Date|CC Security Code|         CC Provider|               Email|                 Job|     IP Address|Language|Purchase Price|
+--------------------+-----+--------+--------------------+--------------------+----------------+-----------+----------------+--------------------+--------------------+--------------------+---------------+--------+--------------+
|7502 Powell Missi...|21 XT|      PM|Mozilla/5.0 (Maci...|      Silva-Anderson|  30246185196287|      07/25|            7169|            Discover|   ynguyen@gmail.com|   Fish farm manager|  55.96.152.147|      ru|         25.15|
|260 Rachel Plains...|96 pG|      PM|Mozilla/5.0 (X11;...|    Marshall-Collins|    5

In [11]:
expensive = commerce.where(commerce['Purchase Price'] > 80.00).where(commerce['Purchase Price'] < 100.00)
expensive.show()

+--------------------+-----+--------+--------------------+--------------------+----------------+-----------+----------------+--------------------+--------------------+--------------------+---------------+--------+--------------+
|             Address|  Lot|AM or PM|        Browser Info|             Company|     Credit Card|CC Exp Date|CC Security Code|         CC Provider|               Email|                 Job|     IP Address|Language|Purchase Price|
+--------------------+-----+--------+--------------------+--------------------+----------------+-----------+----------------+--------------------+--------------------+--------------------+---------------+--------+--------------+
|16629 Pace Camp A...|46 in|      PM|Opera/9.56.(X11; ...|     Martinez-Herman|6011929061123406|      02/20|             900|        JCB 16 digit|   pdunlap@yahoo.com|Scientist, produc...|149.146.147.205|      el|         98.14|
|93971 Conway Caus...|96 Xt|      AM|Mozilla/5.0 (comp...|     Gibson and Sons|60113

# Making job buckets
Now we have to make buckets for all of our unique jobs. We can do this taking the job name out of the list of unique jobs and placing it in a bucket list. If we do this for every occupation then we should be good.

In [12]:
jobs = commerce.select("Job").distinct().collect()
job_list = ["Job List"]
print(f"{len(jobs)} jobs")
for job in jobs:
  print(job[0])
  job_list.append(job[0])

623 jobs
Retail merchandiser
Engineer, aeronautical
Catering manager
Librarian, academic
Diplomatic Services operational officer
Designer, ceramics/pottery
Occupational hygienist
Early years teacher
Patent examiner
English as a second language teacher
Primary school teacher
Control and instrumentation engineer
Clinical molecular geneticist
Estate agent
Art therapist
Transport planner
Applications developer
Petroleum engineer
Conservator, furniture
Stage manager
Investment banker, operational
Adult nurse
Sports administrator
Administrator, Civil Service
Counsellor
Chartered public finance accountant
Personal assistant
Pharmacist, hospital
Journalist, newspaper
Financial manager
Theatre director
Child psychotherapist
Corporate investment banker
Data processing manager
Hydrogeologist
Police officer
Accountant, chartered public finance
Holiday representative
IT sales professional
Ship broker
Copy
Social research officer, government
Geophysicist/field seismologist
TEFL teacher
Education off

# We'll start with a finance bucket.

In [13]:
def bucket_stats(bucket):
  bucket_df = commerce.where(commerce.Job.isin(bucket))
  print(f"Bucket info:")
  print(f"""# transactions: {bucket_df.count()}
  Highest transaction: {int(float(bucket_df.agg({"Purchase Price" : "max"}).collect()[0][0]))}
  Total transaction value: {int(bucket_df.agg({"Purchase Price" : "sum"}).collect()[0][0])}
  """)
  bucket_df.show()

In [14]:
from collections import defaultdict

In [17]:
narrowed_list = job_list
buckets = {}
while narrowed_list:
  temp = defaultdict(int)

  for sub in narrowed_list:
    for wrd in sub.split():
      temp[wrd] += 1

  res = max(temp, key=temp.get).lower().strip(",")

  print(res)

  buckets[f"{res}_list"] = res_list = [job for job in narrowed_list if str(res) in job.lower()]
  print(res_list)
  narrowed_list = list(set(narrowed_list) - set(res_list))
  print(f"{len(narrowed_list)} rows left to sort")

manager
['Catering manager', 'Stage manager', 'Financial manager', 'Data processing manager', 'Housing manager/officer', 'Tour manager', 'Fish farm manager', 'Health service manager', 'Retail manager', 'Hotel manager', 'Environmental manager', 'Restaurant manager', 'Theatre stage manager', 'Art gallery manager', 'Tourist information centre manager', 'Location manager', 'Risk manager', 'Fast food restaurant manager', 'Customer service manager', 'Quality manager', 'Records manager', 'Pension scheme manager', 'Conference centre manager', 'Heritage manager', 'Forest/woodland manager', 'Publishing rights manager', 'Warehouse manager', 'Logistics and distribution manager', 'Theatre manager', 'Accommodation manager', 'Public house manager', 'Fitness centre manager', 'Outdoor activities/education manager', 'Insurance account manager', 'Leisure centre manager', 'Farm manager', 'Estate manager/land agent', 'Theme park manager', 'Purchasing manager', 'Production manager', 'Passenger transport man

In [ ]:
for i in buckets['manager_list']:
  print(i)

In [ ]:
bucket_stats(buckets["manager_list"])